<a href="https://colab.research.google.com/github/faraaz-lab/rsa_chatbot/blob/master/rsa_chatbot_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!mkdir "/content/drive/My Drive/rsa_chatbot_2"
!cd "/content/drive/My Drive/rsa_chatbot_2"


Conversation AI with the Rasa Stack

Part 1: Having a basic conversation

Get Rasa Core

First we'll have to install Rasa Core on the server running this notebook.

In [7]:
!pip install rasa_core;

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 5.4MB 7.6MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 40kB 5.4MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 409kB 41.5MB/s 
     |████████████████████████████████| 235kB 39.9MB/s 
     |████████████████████████████████| 327kB 28.2MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 665kB 42.4MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 92.6MB 103kB/s 
     |████████████████████████████████| 153kB 42.0MB/s 
     |████████████████████████████████| 51kB 2.4MB/s 
     |████████████████████████████████| 112kB 44.3MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 204kB 41.0MB/s 
     |████████████████████████████████| 358kB 39.3MB/s 
     |██

Understanding Conversations through Stories

Lets us get started by writing a few stories, which are example conversations that Rasa Core will learn from.

The format works like this:

A story starts with ## and you can give it a name.

Intents: Lines that start with * are messages sent by the user. Intent (and the entities) that represent what the user means.

Actions: Lines that start with - are actions taken by your bot. In this case all of our actions are just messages sent back to the user, like utter_greet, but in general an action can do anything, including calling an API and interacting with the outside world.

In [3]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy
  
  

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_deny
  - utter_goodbye
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

"""

%store stories_md > "/content/drive/My Drive/rsa_chatbot_2/stories.md"

Writing 'stories_md' (str) to file '/content/drive/My Drive/rsa_chatbot_2/stories.md'.


Defining a Domain for your agent

The domain specifies the universe that your bot lives in. All of the intents and actions that show up in your stories should be listed along with templates, which contain the messages your bot can send back.

In [4]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy

actions:
  - utter_greet
  - utter_cheer_up
  - utter_did_that_help
  - utter_happy
  - utter_goodbye
  - utter_unclear

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    text: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
"""

%store domain_yml > "/content/drive/My Drive/rsa_chatbot_2/domain.yml"

Writing 'domain_yml' (str) to file '/content/drive/My Drive/rsa_chatbot_2/domain.yml'.


In [11]:
choco install python3 --version 3.6.6
# choco install python --pre 

SyntaxError: ignored

Training your Dialogue Model

Now comes the fun part! We're going to show Rasa Core the stories we wrote above, and train a model on these examples.

In this case, the model is a neural network implemented in Keras which learns to predict which action to take next. The default architecture is based on an LSTM.

In [13]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow is already loaded. Please restart the runtime to change versions.
2.1.0


In [9]:
#!mkdir "/content/drive/My Drive/rsa_chatbot_2/models"
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

agent = Agent('/content/drive/My Drive/rsa_chatbot_2/domain.yml', policies=[KerasPolicy()])
training_data = agent.load_data('/content/drive/My Drive/rsa_chatbot_2/stories.md')
agent.train(
        training_data,
        validation_split=0.0
)

agent.persist('/content/drive/My Drive/rsa_chatbot_2/models/dialogue')

ModuleNotFoundError: ignored